<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow-project/FineTuning
#%cd FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 424, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 424 (delta 37), reused 3 (delta 3), pack-reused 364
Receiving objects: 100% (424/424), 93.42 MiB | 14.00 MiB/s, done.
Resolving deltas: 100% (303/303), done.


In [2]:
import sys
import os

py_file_location = "/content/FineTuning"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

In [4]:
from numpy import dot
from numpy.linalg import norm
import os
from google.colab import drive
from PIL import Image
import textual_inversion as txt
#from textual_inversion import train_ds

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
8810/8810 [==============================] - 0s 0us/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


198180272/198180272 [==============================] - 1s 0us/step
[[TensorShape([49409, 768])], [], []]
136824240/136824240 [==============================] - 1s 0us/step


In [6]:
sticker_embedding = []
cosine_similarity = []
broccoli = txt.get_embedding("broccoli")
cosine_similarity.append(txt.cosine_sim(broccoli, txt.get_embedding(txt.placeholder_token)))

In [ ]:
epochs = 5
epoch_num = list(range(epochs+1))
txt.training(epoch=epochs, model=txt.stable_diffusion, data= txt.train_ds)

In [ ]:
plt.plot(epoch_num, cosine_similarity)
plt.xlabel("Number of Epochs")
plt.ylabel("Cosine Similarity")
plt.title("Cosine Similarity between the basis and the new concept")
plt.show()

In [ ]:
drive.mount("/content/drive")
path = '/content/drive/MyDrive/angryweeeights.npy'

In [ ]:
###get the weights of the text encoder and save the to google drive
text_encoder_weights = np.array(stable_diffusion.text_encoder.get_weights())

### Save the weights array to a file on your Google Drive
np.save(path, text_encoder_weights)

In [ ]:
###load the array of the weights of the text encoder from the last training from google drive
text_encoder_weights = np.load(path, allow_pickle=True)

### Set the weights of the text encoder
stable_diffusion.text_encoder.set_weights(text_encoder_weights)

In [ ]:
import shutil
drive_folder = '/content/drive/MyDrive/angrybildeeer/'

### get the number, we stopped the last time to name our pictures, to ensure each picture gets a different name
i_file = os.path.join(drive_folder, 'i.txt')
if os.path.isfile(i_file):
    with open(i_file, 'r') as f:
        i = int(f.read())
else:
    i = 0

for j in range(10):
    generated = stable_diffusion.text_to_image(
    f" a angry {placeholder_token}.", batch_size=1,  num_steps=25 )
    broc = generated[0]

    ### convert the array generated from our stable diffusion model into a picture
    broc = Image.fromarray(broc, mode='RGB')

    broc.save(f'image_{i}.jpg')

    ### save the picture to google drive
    local_path = f'image_{i}.jpg'
    drive_path = os.path.join(drive_folder, f'image_{i}.jpg')  # Use f-string to include variable in file name
    shutil.copy(local_path, drive_path)

    ### store the value of i in the file, to ensure no picture will have the same name
    i += 1
    with open(i_file, 'w') as f:
        f.write(str(i))